In [ ]:
!pip install pydriller==2.6 gitpython transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.0 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/1200wd/bitcoinlib
repo_path = "/content/bitcoinlib"


Cloning into 'bitcoinlib'...
remote: Enumerating objects: 22726, done.
remote: Counting objects: 100% (4141/4141), done.
remote: Compressing objects: 100% (550/550), done.
remote: Total 22726 (delta 3695), reused 3591 (delta 3591), pack-reused 18585 (from 3)
Receiving objects: 100% (22726/22726), 11.02 MiB | 17.38 MiB/s, done.
Resolving deltas: 100% (17740/17740), done.


In [ ]:
import pandas as pd
from pydriller import Repository

bug_commits = []

for commit in Repository(path_to_repo=repo_path).traverse_commits():
    msg_lower = commit.msg.lower()
    if any(word in msg_lower for word in ["fix", "bug", "error", "issue", "patch"]):
        bug_commits.append({
            "hash": commit.hash,
            "message": commit.msg,
            "parents": commit.parents,
            "is_merge": commit.merge,
            "modified_files": [m.new_path for m in commit.modified_files if m.new_path]
        })

df_bug = pd.DataFrame(bug_commits)
df_bug.to_csv("bug_commits.csv", index=False)
df_bug


,hash,message,parents,is_merge,modified_files
0,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",[7d6b8e9ab3e738d69e39ad53a7c0f6a552d5284a],False,"[artwork/logo-full.svg, docs/_static/flask.png..."
1,8e9bd5f009bb8756a2a7d76a4b792323eeee49d6,Fixed a typo,[9dfab582d065521e9350327801ed2c5ced1c8fa1],False,[docs/index.rst]
2,1fff3e598126a084348ec2c112fdd3bc6b9a1ee0,Fixed a doc display bug and setup.py workaroun...,[05f36c7f7e2df36ee28f90d99632162579b1287c],False,"[docs/_themes/flasky/static/flasky.css_t, docs..."
3,574e81f9c8bbdc41958e1e7a7613633b091101f8,Fixed a bug in setup.py,[1fff3e598126a084348ec2c112fdd3bc6b9a1ee0],False,"[docs/quickstart.rst, setup.py]"
4,2f5a4f8dbc832b0daebcd66ea8b3969589191fa7,Doc updates and typo fixes,[03148dba6b26a2694da1d466584f50189c52e7b3],False,"[docs/testing.rst, examples/minitwit/README, e..."
...,...,...,...,...,...
1732,73d6504063bfa00666a92b07a28aaf906c532f09,Merge commit from fork\n\nSessions: fix signin...,"[941efd4a36ed0f27e13758874f95e3aa1d3ee163, cbb...",True,[]
1733,bbaf13333fd00644313488243cfe547b13dcae78,fix syntax,[57e72869486e89b45f6fd0a07cdd0adf80f5db1b],False,[CHANGES.rst]
1734,184ec3c5453e1156372085df35e8d922ce7e3559,Fix global CONTRIBUTING link,[bbaf13333fd00644313488243cfe547b13dcae78],False,[docs/contributing.rst]
1735,a42c4d54a3e5cd56c0700479ede75157a748acd6,Fix global CONTRIBUTING link (#5737),"[bbaf13333fd00644313488243cfe547b13dcae78, 184...",True,[]


In [ ]:
rows = []
for commit in Repository(path_to_repo=repo_path).traverse_commits():
    if commit.hash in df_bug["hash"].values:
        for m in commit.modified_files:
            try:
                source_before = m.source_code_before or ""
            except Exception:
                source_before = ""

            try:
                source_current = m.source_code or ""
            except Exception:
                source_current = ""

            rows.append({
                "hash": commit.hash,
                "message": commit.msg,
                "filename": m.new_path,
                "source_before": source_before,
                "source_current": source_current,
                "diff": getattr(m, "diff", "") or str(getattr(m, "diff_parsed", ""))
            })

df_diff = pd.DataFrame(rows)
df_diff.to_csv("bug_diffs.csv", index=False)
df_diff.head()


,hash,message,filename,source_before,source_current,diff
0,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",artwork/logo-full.svg,,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...","@@ -0,0 +1,329 @@\n+<?xml version=""1.0"" encodi..."
1,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_static/flask.png,,PNG\r\n\n   \rIHDR         n֕   sBIT|...,Binary files /dev/null and b/docs/_static/flas...
2,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_static/logo-full.png,,PNG\r\n\n   \rIHDR        >j9   sBIT|...,Binary files /dev/null and b/docs/_static/logo...
3,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_templates/sidebarintro.html,,<h3>About Flask</h3>\n<p>\n Flask is a micro ...,"@@ -0,0 +1,7 @@\n+<h3>About Flask</h3>\n+<p>\n..."
4,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_templates/sidebarlogo.html,,"<p class=""logo""><a href=""{{ pathto(master_doc)...","@@ -0,0 +1,3 @@\n+<p class=""logo""><a href=""{{ ..."


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "mamiksik/CommitPredictorT5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
def predict_message(diff_text):
    inputs = tokenizer(diff_text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example
df_diff["LLM_inference"] = df_diff["diff"].apply(lambda d: predict_message(d[:4000]) if pd.notna(d) else "")
df_diff.to_csv("bug_diffs_with_llm.csv", index=False)
df_diff.head()

,hash,message,filename,source_before,source_current,diff,LLM_inference
0,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",artwork/logo-full.svg,,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...","@@ -0,0 +1,329 @@\n+<?xml version=""1.0"" encodi...",add new example in inkscape
1,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_static/flask.png,,PNG\r\n\n   \rIHDR         n֕   sBIT|...,Binary files /dev/null and b/docs/_static/flas...,fix flask image
2,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_static/logo-full.png,,PNG\r\n\n   \rIHDR        >j9   sBIT|...,Binary files /dev/null and b/docs/_static/logo...,fix logo path
3,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_templates/sidebarintro.html,,<h3>About Flask</h3>\n<p>\n Flask is a micro ...,"@@ -0,0 +1,7 @@\n+<h3>About Flask</h3>\n+<p>\n...",add link to the missing word in the list
4,3d719f35f5c1ee4ce3dc01fb2167ef49c0180cf6,"Added docs, fixed some bugs I introduced last ...",docs/_templates/sidebarlogo.html,,"<p class=""logo""><a href=""{{ pathto(master_doc)...","@@ -0,0 +1,3 @@\n+<p class=""logo""><a href=""{{ ...",add logo to missing missing tag
